In [2]:
import pandas as pd
from sqlalchemy import create_engine
import matplotlib.pyplot as plt

# ---------- Conexão com o banco ----------
user = "postgres"
password = "capoeira"
host = "localhost"
port = 5432
database = "analise_de_dados"

engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}')

# ---------- Carregar tabelas ----------
df_projects = pd.read_sql("SELECT * FROM Projetos;", engine)
df_tasks = pd.read_sql("SELECT * FROM Tarefas;", engine)
df_employees = pd.read_sql("SELECT * FROM Empregados;", engine)
df_logs = pd.read_sql("SELECT * FROM Logs;", engine)
df_costs = pd.read_sql("SELECT * FROM Custos;", engine)
df_resources = pd.read_sql("SELECT * FROM Recursos;", engine)
df_project_resources = pd.read_sql("SELECT * FROM Recursos_projetos;", engine)

# ---------- Funções do sistema ----------
def listar_projetos():
    print("\nLista de Projetos:")
    display(df_projects[['projeto_id', 'nome', 'descricao', 'data_inicio', 'data_fim', 'status']])

def recursos_por_projeto():
    projeto_id = int(input("Digite o ID do projeto: "))
    projeto = df_projects[df_projects['projeto_id'] == projeto_id]
    if projeto.empty:
        print("Projeto não encontrado!")
        return

    print(f"\nRecursos do Projeto: {projeto['nome'].values[0]}")
    df_proj_res = df_project_resources[df_project_resources['projeto_id'] == projeto_id]
    df_rec = df_proj_res.merge(df_resources, left_on='recurso_id', right_on='recurso_id')
    df_rec['custo_total'] = df_rec['quantidade'] * df_rec['custo_unidade']
    display(df_rec[['nome', 'tipo', 'quantidade', 'custo_total']])
    print(f"Custo total de recursos: R$ {df_rec['custo_total'].sum():.2f}")

def tarefas_atrasadas():
    atrasadas = df_tasks[df_tasks['status'] == 'atrasado']
    print("\nTarefas Atrasadas:")
    display(atrasadas[['tarefas_id', 'nome', 'projeto_id', 'id_responsavel', 'data_fim']])

def grafico_custos():
    df_proj_res = df_project_resources.merge(df_resources, left_on='recurso_id', right_on='recurso_id')
    df_proj_res['custo_unitario'] = df_proj_res['quantidade'] * df_proj_res['custo_unitario']
    custos = df_proj_res.groupby('projeto_id')['custo_unitario'].sum().reset_index()
    custos = custos.merge(df_projects, left_on='projeto_id', right_on='projeto_id')

    plt.figure(figsize=(8,5))
    plt.bar(custos['nome'], custos['custo_unitario'], color='skyblue')
    plt.ylabel("Custo total (R$)")
    plt.title("Custo de Recursos por Projeto")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

# ---------- Menu interativo ----------
def menu():
    while True:
        print("\n=== Sistema de Análise de Projetos ===")
        print("1. Listar projetos")
        print("2. Visualizar recursos por projeto")
        print("3. Ver tarefas atrasadas")
        print("4. Gráfico de custos por projeto")
        print("0. Sair")
        escolha = input("Escolha uma opção: ")

        if escolha == '1':
            listar_projetos()
        elif escolha == '2':
            recursos_por_projeto()
        elif escolha == '3':
            tarefas_atrasadas()
        elif escolha == '4':
            grafico_custos()
        elif escolha == '0':
            print("Saindo do sistema...")
            break
        else:
            print("Opção inválida! Tente novamente.")

# ---------- Rodar sistema ----------
menu()



=== Sistema de Análise de Projetos ===
1. Listar projetos
2. Visualizar recursos por projeto
3. Ver tarefas atrasadas
4. Gráfico de custos por projeto
0. Sair


Escolha uma opção:  1



Lista de Projetos:


,projeto_id,nome,descricao,data_inicio,data_fim,status



=== Sistema de Análise de Projetos ===
1. Listar projetos
2. Visualizar recursos por projeto
3. Ver tarefas atrasadas
4. Gráfico de custos por projeto
0. Sair


Escolha uma opção:  0


Saindo do sistema...
